# Before your start:
- Read the README.md file
- Comment as much as you can and use the resources (README.md file)
- Happy learning!

In [117]:
import pandas as pd
import numpy as np

# Challenge 1 - Index Functions

In this challenge, we will practice the advanced functions introduced in the lesson. However, before we start, we will load and evaluate our dataset. 

Load the dataset from Ironhack's database:
* db: `admission_predict`
* table: `admission_predict`

In [118]:
df = pd.read_csv('admission_predict.csv')

Remove trailing spaces at the end of the column names if there are any.


In [119]:
df = df.rename(columns=lambda x: x.strip())

Let's evaluate the dataset by looking at the `head` function.

In [120]:
df.head()

,Serial No.,GRE Score,TOEFL Score,University Rating,SOP,LOR,CGPA,Research,Chance of Admit
0,1,337,118,4,4.5,4.5,9.65,1,0.92
1,2,316,104,3,3.0,3.5,8.00,1,0.72
2,3,322,110,3,3.5,2.5,8.67,1,0.80
3,4,314,103,2,2.0,3.0,8.21,0,0.65
4,5,330,115,5,4.5,3.0,9.34,1,0.90


Before beginning to work with this dataset and evaluating graduate admissions data, we will verify that there is no missing data in the dataset. Do this in the cell below.

In [121]:
df.isna().sum()

Serial No.           0
GRE Score            0
TOEFL Score          0
University Rating    0
SOP                  0
LOR                  0
CGPA                 0
Research             0
Chance of Admit      0
dtype: int64

Interestingly, there is a column that uniquely identifies the applicants. This column is the serial number column. Instead of having our own index, we should make this column our index. Do this in the cell below. Keep the column in the dataframe in addition to making it an index.

In [122]:
df = df.set_index('Serial No.')

In [123]:
df.head()

,GRE Score,TOEFL Score,University Rating,SOP,LOR,CGPA,Research,Chance of Admit
Serial No.,,,,,,,,
1,337,118,4,4.5,4.5,9.65,1,0.92
2,316,104,3,3.0,3.5,8.00,1,0.72
3,322,110,3,3.5,2.5,8.67,1,0.80
4,314,103,2,2.0,3.0,8.21,0,0.65
5,330,115,5,4.5,3.0,9.34,1,0.90


Turns out that `GRE Score` and `CGPA` also uniquely identify the data. Show this in the cell below.

In [124]:
sum(df[['GRE Score', 'CGPA']].duplicated())

0

Replace the index with an index comprised of two columns - `GRE Score` and `CGPA`. Remove the columns from the dataset as well. Make sure to use `inplace=True`.

Now change the index back to a sequence starting at zero using the `reset_index` function. Make sure to use `inplace=True`.

In [125]:
df.set_index(['GRE Score','CGPA'], inplace = True)

In [126]:
df.reset_index(inplace = True)

In [127]:
df.head()

,GRE Score,CGPA,TOEFL Score,University Rating,SOP,LOR,Research,Chance of Admit
0,337,9.65,118,4,4.5,4.5,1,0.92
1,316,8.00,104,3,3.0,3.5,1,0.72
2,322,8.67,110,3,3.5,2.5,1,0.80
3,314,8.21,103,2,2.0,3.0,0,0.65
4,330,9.34,115,5,4.5,3.0,1,0.90


# Challenge 2 - Advanced Functions

In this part of the lab, we would like to test complex conditions on the entire dataset at once. Let's start by finding the number of rows where the CGPA is higher than 9 and the student has conducted research.

In [128]:
len(df[(df['CGPA'] > 9) & (df['Research'] == 1)])

101

Now return all the rows where the CGPA is greater than 9 and the SOP score is less than 3.5. Find the mean chance of admit for these applicants.

In [129]:
len(df[(df['CGPA'] > 9) & (df['SOP'] < 3.5)])

5

In [130]:
df[(df['CGPA'] > 9) & (df['SOP'] < 3.5)]['Chance of Admit'].mean()

0.8019999999999999

We would like to create a deciding factor column for each student. We standardize several columns and then pick the most important factor from a lookup table. If the standardized value is above 0.8, the student will be accepted.

We will start by creating a standardized column for `CGPA`, `GRE Score` and `LOR`. We will name these columns `CGPA_std`, `GRE_std`, and `LOR_std` respecively.

Recall that standardizing a column is done by subtracting the mean of the column from all observations in the column and then dividing each observation in the column by the column's standard deviation.

In the cell below, write a standardization function (a function that takes a column as input and returns a standardized column as output). Make sure to use the `numpy` versions of mean and standard deviation.

In [131]:
def standardize(col):
    """
    This function takes a column from a dataframe 
    and returns a standardized column by subtracting the column's mean
    and dividing by the column's standard deviation.
    """
    
    return ((col - col.mean())/col.std())

Now create the standardized columns `CGPA_std`, `GRE_std`, and `LOR_std` and add them to the `admissions` dataframe.

In [132]:
df['CGPA_std'] = standardize(df['CGPA'])
df['GRE_std'] = standardize(df['GRE Score'])
df['LOR_std'] = standardize(df['LOR'])

We will generate the decision choice at random using the code below. Please run the cell.

In [133]:
# Libraries
from random import choices

In [134]:
std_columns = ['CGPA_std', 'GRE_std', 'LOR_std']

decision_choice = choices(std_columns, k=df.shape[0])

Now create the deciding column using the `lookup` function. The lookup column is `decision_choice` found above. Call the column resulting from the lookup function `deciding_column` and add it to the `admissions` dataframe.

In [135]:
df['deciding_column'] = df.lookup(df.index, decision_choice)

In [136]:
df.head()

,GRE Score,CGPA,TOEFL Score,University Rating,SOP,LOR,Research,Chance of Admit,CGPA_std,GRE_std,LOR_std,deciding_column
0,337,9.65,118,4,4.5,4.5,1,0.92,1.747900,1.753381,1.191647,1.753381
1,316,8.00,104,3,3.0,3.5,1,0.72,-0.991211,-0.063367,0.076740,0.076740
2,322,8.67,110,3,3.5,2.5,1,0.80,0.121034,0.455704,-1.038166,-1.038166
3,314,8.21,103,2,2.0,3.0,0,0.65,-0.642597,-0.236391,-0.480713,-0.236391
4,330,9.34,115,5,4.5,3.0,1,0.90,1.233279,1.147798,-0.480713,1.233279


Create a column called `decision` in the `admissions` dataframe. Assign 1 to this column if the value of `deciding_column` is greater than 0.8 and 0 otherwise.

In [137]:
df['decision'] = np.where(df.deciding_column > 0.8, 1, 0)

How many applicants will be accepted to the program using the decision column? Compute the result below.

In [138]:
df['decision'].sum()

86

# Challenge 3 - Method Chaining

To increase our coding efficiency, let's make a number of changes to our dataframe in one line of code.

In the cell below, remove all non character symbols from the column names, replace all spaces in column names to underscores, and change all upper case characters to lower case. Assign these new column values to `admissions.columns`.

In [139]:
df.columns = df.columns.str.replace(' ','_')
df.columns = df.columns.str.lstrip()
df.columns = df.columns.str.rstrip()
df.columns = df.columns.str.replace('\s+', '_').str.replace('\W+', '')
df.columns = df.columns.str.lower()

In [143]:
df.head()

,gre_score,cgpa,toefl_score,university_rating,sop,lor,research,chance_of_admit,cgpa_std,gre_std,lor_std,deciding_column,decision
0,347,9.65,118,4,4.5,4.5,1,0.92,1.747900,1.753381,1.191647,1.753381,1
1,326,8.00,104,3,3.0,3.5,1,0.72,-0.991211,-0.063367,0.076740,0.076740,0
2,332,8.67,110,3,3.5,2.5,1,0.80,0.121034,0.455704,-1.038166,-1.038166,0
3,324,8.21,103,2,2.0,3.0,0,0.65,-0.642597,-0.236391,-0.480713,-0.236391,0
4,340,9.34,115,5,4.5,3.0,1,0.90,1.233279,1.147798,-0.480713,1.233279,1


In the cell below, give all student with a university rating of 4 or higher a 10 point boost on their GRE score and split the column into 4 bins using the `cut` function. Assign this new score to the variable `adjusted_gre`.

In [144]:
df.gre_score += 10

In [145]:
df.head()

,gre_score,cgpa,toefl_score,university_rating,sop,lor,research,chance_of_admit,cgpa_std,gre_std,lor_std,deciding_column,decision
0,357,9.65,118,4,4.5,4.5,1,0.92,1.747900,1.753381,1.191647,1.753381,1
1,336,8.00,104,3,3.0,3.5,1,0.72,-0.991211,-0.063367,0.076740,0.076740,0
2,342,8.67,110,3,3.5,2.5,1,0.80,0.121034,0.455704,-1.038166,-1.038166,0
3,334,8.21,103,2,2.0,3.0,0,0.65,-0.642597,-0.236391,-0.480713,-0.236391,0
4,350,9.34,115,5,4.5,3.0,1,0.90,1.233279,1.147798,-0.480713,1.233279,1


In [146]:
df['adjusted_gre'] = pd.cut(df['gre_score'], 4)

In [151]:
df.head()

,gre_score,cgpa,toefl_score,university_rating,sop,lor,research,chance_of_admit,cgpa_std,gre_std,lor_std,deciding_column,decision,adjusted_gre
0,357,9.65,118,4,4.5,4.5,1,0.92,1.747900,1.753381,1.191647,1.753381,1,"(347.5, 360.0]"
1,336,8.00,104,3,3.0,3.5,1,0.72,-0.991211,-0.063367,0.076740,0.076740,0,"(335.0, 347.5]"
2,342,8.67,110,3,3.5,2.5,1,0.80,0.121034,0.455704,-1.038166,-1.038166,0,"(335.0, 347.5]"
3,334,8.21,103,2,2.0,3.0,0,0.65,-0.642597,-0.236391,-0.480713,-0.236391,0,"(322.5, 335.0]"
4,350,9.34,115,5,4.5,3.0,1,0.90,1.233279,1.147798,-0.480713,1.233279,1,"(347.5, 360.0]"
